# 🌊 Free FLUX.1 dev on Google Colab (ComfyUI)

**Run Black Forest Labs' FLUX.1 dev model via ComfyUI — for free on Google Colab T4 GPU.**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

---

## Why FLUX.1 dev beats Stable Diffusion

| Feature | SD 1.5 / SDXL | FLUX.1 dev |
|---------|--------------|------------|
| Parameters | 860M / 3.5B | **12B** |
| Prompt adherence | Moderate | **Excellent** |
| Text in images | Poor | **Good** |
| Hands / anatomy | Needs ADetailer | **Much better natively** |
| Realism | Good (fine-tuned) | **Excellent out of the box** |
| Architecture | UNet | **Diffusion Transformer (DiT)** |
| License | Open / Apache 2.0 | Non-commercial (research) |

## Setup (one-time, ~6 minutes)

1. **File → Upload notebook** → select this file
2. **Runtime → Change runtime type** → Hardware accelerator: **T4 GPU** → **Save**
3. **Runtime → Run all** — ~6 min total (downloads ~12 GB of models)
4. **Click the `trycloudflare.com` URL** in Cell 3 output → ComfyUI opens!

> **No HuggingFace token required!** All models are downloaded from public mirrors.

### What's included
- **FLUX.1 dev GGUF Q5** — 12B parameter DiT model, Q5 quantized (~8.6 GB) — works great on T4!
- **ComfyUI** — node-based interface with native FLUX support
- **ComfyUI-Manager** — install extra nodes from within the UI
- **ComfyUI-GGUF** — custom node for loading GGUF quantized models
- **T5-XXL fp8 + CLIP-L** text encoders (FLUX uses dual encoders)
- **Cloudflare Tunnel** — reliable public URL each session
- Pre-built FLUX workflow with all settings configured

### Why GGUF instead of fp8?
- **fp8 models hang on T4** — T4 lacks native fp8 hardware support (needs Ampere/Ada GPUs)
- **GGUF Q5 works reliably** — Tested extensively on T4, no hanging or CUDA errors
- **Near-identical quality** — Q5 quantization preserves 99%+ of fp8 quality
- **Smaller download** — 8.6 GB vs 12 GB for fp8

### T4 VRAM breakdown
| Component | Size | Location |
|-----------|------|----------|
| FLUX.1 dev GGUF Q5 | ~8.6 GB | GPU (with `--lowvram` offloading) |
| T5-XXL fp8 | ~4.9 GB | CPU (offloaded) |
| CLIP-L | ~246 MB | GPU |
| VAE (ae) | ~335 MB | GPU |

> **Note:** `--lowvram` is used so ComfyUI streams layers to GPU as needed. Generation takes ~60-90 sec on T4.

### Known limitations
- Free Colab sessions last ~12 hours — **save your images!**
- ~60–90 sec/image at 1024×1024 on T4 (vs ~30 sec for SD 1.5)
- FLUX.1 dev is **non-commercial** (research/personal use only)
- The public URL changes every session

---

In [ ]:
# ============================================================
# Cell 1: GPU check + clone ComfyUI + install dependencies
# Runtime: ~3-4 minutes
# ============================================================

print("=" * 60)
print("  CELL 1: Installing ComfyUI + dependencies")
print("=" * 60)

# ── GPU check ──
print("\n🔍 GPU check:")
!nvidia-smi -L
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# ── Clone ComfyUI ──
print("\n📦 Cloning ComfyUI...")
!rm -rf /content/ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI

# ── Install ComfyUI requirements ──
print("\n📦 Installing ComfyUI requirements...")
%cd /content/ComfyUI
!pip install -q -r requirements.txt

# ── Verify PyTorch CUDA (don't override Colab's pre-installed version) ──
print("\n📦 Checking PyTorch CUDA...")
import torch
if torch.cuda.is_available():
    print(f"  ✅ PyTorch {torch.__version__} with CUDA already installed")
else:
    print("  ⚠️  CUDA not available, installing PyTorch with CUDA...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# ── Clone ComfyUI-Manager (install extra nodes from UI) ──
print("\n📦 Installing ComfyUI-Manager...")
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git \
    /content/ComfyUI/custom_nodes/ComfyUI-Manager
!pip install -q -r /content/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt 2>/dev/null || true

# ── Install ComfyUI-GGUF (required for GGUF quantized models) ──
print("\n📦 Installing ComfyUI-GGUF...")
!pip install -q --upgrade gguf
!git clone https://github.com/city96/ComfyUI-GGUF.git \
    /content/ComfyUI/custom_nodes/ComfyUI-GGUF

# ── Verify PyTorch CUDA ──
import torch
print(f"\n✅ PyTorch {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ VRAM: {vram:.1f} GB")

print("\n" + "=" * 60)
print("  ✅ CELL 1 COMPLETE — Now run Cell 2")
print("=" * 60)

In [ ]:
# ============================================================
# Cell 2: Download FLUX.1 dev models (GGUF quantized for T4)
# Runtime: ~3-4 minutes (~12 GB total)
# ============================================================
# Files downloaded:
#   flux1-dev-Q5_K_S.gguf       (~8.6 GB) — FLUX.1 dev, Q5 quantized (T4 compatible!)
#   t5xxl_fp8_e4m3fn.safetensors (~4.9 GB) — T5-XXL text encoder, fp8
#   clip_l.safetensors          (~246 MB) — OpenAI CLIP-L
#   ae.safetensors              (~335 MB) — FLUX VAE
#
# ⚠️  Why GGUF instead of fp8?
#     - fp8 models hang on T4 due to missing CUDA fp8 hardware support
#     - GGUF Q5 models work reliably on T4 and produce near-identical quality
#     - Smaller download, faster loading, same great results!
# ============================================================

print("=" * 60)
print("  CELL 2: Downloading FLUX.1 dev models (GGUF for T4)")
print("=" * 60)

import os

COMFY = "/content/ComfyUI"
UNET_DIR  = f"{COMFY}/models/unet"
CLIP_DIR  = f"{COMFY}/models/clip"
VAE_DIR   = f"{COMFY}/models/vae"

os.makedirs(UNET_DIR, exist_ok=True)
os.makedirs(CLIP_DIR, exist_ok=True)
os.makedirs(VAE_DIR,  exist_ok=True)

def check_file(path, min_bytes, name):
    """Check if file exists and meets minimum size. Delete if corrupted."""
    if os.path.exists(path):
        sz = os.path.getsize(path)
        if sz >= min_bytes:
            print(f"  ✅ {name} already present ({sz / 1024**2:.0f} MB)")
            return True
        else:
            print(f"  ⚠️  {name} is too small ({sz / 1024**2:.1f} MB) — corrupted/incomplete, re-downloading...")
            os.remove(path)
    return False

# ── FLUX.1 dev GGUF Q5 (city96's quantized version — works great on T4!) ──
# Source: https://huggingface.co/city96/FLUX.1-dev-gguf
FLUX_DEV_GGUF = f"{UNET_DIR}/flux1-dev-Q5_K_S.gguf"
if not check_file(FLUX_DEV_GGUF, 8_000_000_000, "FLUX.1 dev Q5 GGUF"):
    print("\n📦 Downloading FLUX.1 dev GGUF Q5 (~8.6 GB) — works great on T4...")
    !wget -q --show-progress -c \
        https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q5_K_S.gguf \
        -O "{FLUX_DEV_GGUF}"

# ── T5-XXL fp8 text encoder ──
T5_PATH = f"{CLIP_DIR}/t5xxl_fp8_e4m3fn.safetensors"
if not check_file(T5_PATH, 4_000_000_000, "T5-XXL fp8"):
    print("\n📦 Downloading T5-XXL fp8 (~4.9 GB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors \
        -O "{T5_PATH}"

# ── CLIP-L ──
CLIP_PATH = f"{CLIP_DIR}/clip_l.safetensors"
if not check_file(CLIP_PATH, 200_000_000, "CLIP-L"):
    print("\n📦 Downloading CLIP-L (~246 MB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors \
        -O "{CLIP_PATH}"

# ── FLUX VAE (ae.safetensors) — using cocktailpeanut's ungated mirror ──
VAE_PATH = f"{VAE_DIR}/ae.safetensors"
if not check_file(VAE_PATH, 300_000_000, "FLUX VAE"):
    print("\n📦 Downloading FLUX VAE (~335 MB)...")
    !wget -q --show-progress -c \
        https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors \
        -O "{VAE_PATH}"

# ── Verify all files with strict size checks ──
print("\n📋 File verification:")
files = {
    "FLUX.1 dev Q5 GGUF": (FLUX_DEV_GGUF, 8_000_000_000),
    "T5-XXL fp8":         (T5_PATH,       4_000_000_000),
    "CLIP-L":             (CLIP_PATH,       200_000_000),
    "FLUX VAE":           (VAE_PATH,        300_000_000),
}
all_ok = True
for name, (path, min_sz) in files.items():
    if os.path.exists(path):
        sz = os.path.getsize(path)
        if sz >= min_sz:
            print(f"  ✅ {name}: {sz / 1024**2:.0f} MB")
        else:
            print(f"  ❌ {name}: {sz / 1024**2:.1f} MB — TOO SMALL (corrupted), re-run this cell")
            all_ok = False
    else:
        print(f"  ❌ {name}: MISSING")
        all_ok = False

if all_ok:
    print("\n" + "=" * 60)
    print("  ✅ CELL 2 COMPLETE — Now run Cell 3")
    print("=" * 60)
else:
    print("\n⚠️  Some files missing or corrupted — re-run this cell")

In [ ]:
# ============================================================
# Cell 3: Launch ComfyUI with FLUX.1 dev workflow (GGUF)
# Runtime: ~1-2 minutes (first launch compiles CUDA kernels)
# ============================================================

print("=" * 60)
print("  CELL 3: Launching ComfyUI with FLUX.1 dev (GGUF)")
print("=" * 60)

import os, json, subprocess, time, re

os.chdir('/content/ComfyUI')

# ================================================================
# Save workflow using UnetLoaderGGUF (from ComfyUI-GGUF custom node).
# This works reliably on T4 GPUs where fp8 models hang.
# ================================================================
print("\n⚙️  Writing FLUX.1 dev GGUF workflow...")

flux_workflow = {
  "last_node_id": 13,
  "last_link_id": 16,
  "nodes": [
    {
      "id": 1, "type": "UnetLoaderGGUF",
      "pos": [50, 50], "size": [315, 58],
      "flags": {}, "order": 0, "mode": 0,
      "outputs": [{"name": "MODEL", "type": "MODEL", "links": [7, 10], "slot_index": 0}],
      "properties": {"Node name for S&R": "UnetLoaderGGUF"},
      "widgets_values": ["flux1-dev-Q5_K_S.gguf"],
      "title": "Load FLUX.1 dev (GGUF Q5)"
    },
    {
      "id": 2, "type": "DualCLIPLoader",
      "pos": [50, 180], "size": [315, 106],
      "flags": {}, "order": 1, "mode": 0,
      "outputs": [{"name": "CLIP", "type": "CLIP", "links": [1], "slot_index": 0}],
      "properties": {"Node name for S&R": "DualCLIPLoader"},
      "widgets_values": ["t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux"],
      "title": "Dual CLIP Loader (T5 + CLIP-L)"
    },
    {
      "id": 3, "type": "VAELoader",
      "pos": [50, 360], "size": [315, 58],
      "flags": {}, "order": 2, "mode": 0,
      "outputs": [{"name": "VAE", "type": "VAE", "links": [12], "slot_index": 0}],
      "properties": {"Node name for S&R": "VAELoader"},
      "widgets_values": ["ae.safetensors"],
      "title": "Load FLUX VAE"
    },
    {
      "id": 4, "type": "CLIPTextEncode",
      "pos": [450, 180], "size": [400, 200],
      "flags": {}, "order": 5, "mode": 0,
      "inputs": [{"name": "clip", "type": "CLIP", "link": 1}],
      "outputs": [{"name": "CONDITIONING", "type": "CONDITIONING", "links": [3], "slot_index": 0}],
      "properties": {"Node name for S&R": "CLIPTextEncode"},
      "widgets_values": ["a hyperrealistic photograph of a woman in a cafe, natural window light, shallow depth of field, 85mm lens, film grain, Sony A7R IV"],
      "title": "Positive Prompt"
    },
    {
      "id": 5, "type": "EmptySD3LatentImage",
      "pos": [450, 450], "size": [315, 106],
      "flags": {}, "order": 3, "mode": 0,
      "outputs": [{"name": "LATENT", "type": "LATENT", "links": [11], "slot_index": 0}],
      "properties": {"Node name for S&R": "EmptySD3LatentImage"},
      "widgets_values": [1024, 1024, 1],
      "title": "Empty Latent (1024x1024)"
    },
    {
      "id": 6, "type": "FluxGuidance",
      "pos": [900, 180], "size": [315, 58],
      "flags": {}, "order": 8, "mode": 0,
      "inputs": [{"name": "conditioning", "type": "CONDITIONING", "link": 3}],
      "outputs": [{"name": "CONDITIONING", "type": "CONDITIONING", "links": [5], "slot_index": 0}],
      "properties": {"Node name for S&R": "FluxGuidance"},
      "widgets_values": [3.5],
      "title": "FLUX Guidance (3.5)"
    },
    {
      "id": 7, "type": "BasicGuider",
      "pos": [900, 310], "size": [222, 46],
      "flags": {}, "order": 9, "mode": 0,
      "inputs": [
        {"name": "model", "type": "MODEL", "link": 7},
        {"name": "conditioning", "type": "CONDITIONING", "link": 5}
      ],
      "outputs": [{"name": "GUIDER", "type": "GUIDER", "links": [8], "slot_index": 0}],
      "properties": {"Node name for S&R": "BasicGuider"},
      "title": "Basic Guider"
    },
    {
      "id": 8, "type": "RandomNoise",
      "pos": [900, 420], "size": [315, 82],
      "flags": {}, "order": 4, "mode": 0,
      "outputs": [{"name": "NOISE", "type": "NOISE", "links": [6], "slot_index": 0}],
      "properties": {"Node name for S&R": "RandomNoise"},
      "widgets_values": [42, "fixed"],
      "title": "Random Noise"
    },
    {
      "id": 9, "type": "KSamplerSelect",
      "pos": [900, 560], "size": [315, 58],
      "flags": {}, "order": 6, "mode": 0,
      "outputs": [{"name": "SAMPLER", "type": "SAMPLER", "links": [9], "slot_index": 0}],
      "properties": {"Node name for S&R": "KSamplerSelect"},
      "widgets_values": ["euler"],
      "title": "Sampler: euler"
    },
    {
      "id": 10, "type": "BasicScheduler",
      "pos": [900, 680], "size": [315, 106],
      "flags": {}, "order": 7, "mode": 0,
      "inputs": [{"name": "model", "type": "MODEL", "link": 10}],
      "outputs": [{"name": "SIGMAS", "type": "SIGMAS", "links": [13], "slot_index": 0}],
      "properties": {"Node name for S&R": "BasicScheduler"},
      "widgets_values": ["simple", 20, 1.0],
      "title": "Scheduler: simple, 20 steps"
    },
    {
      "id": 11, "type": "SamplerCustomAdvanced",
      "pos": [1300, 280], "size": [272, 124],
      "flags": {}, "order": 10, "mode": 0,
      "inputs": [
        {"name": "noise", "type": "NOISE", "link": 6},
        {"name": "guider", "type": "GUIDER", "link": 8},
        {"name": "sampler", "type": "SAMPLER", "link": 9},
        {"name": "sigmas", "type": "SIGMAS", "link": 13},
        {"name": "latent_image", "type": "LATENT", "link": 11}
      ],
      "outputs": [
        {"name": "output", "type": "LATENT", "links": [14], "slot_index": 0},
        {"name": "denoised_output", "type": "LATENT", "links": None}
      ],
      "properties": {"Node name for S&R": "SamplerCustomAdvanced"},
      "title": "Sampler"
    },
    {
      "id": 12, "type": "VAEDecode",
      "pos": [1650, 280], "size": [210, 46],
      "flags": {}, "order": 11, "mode": 0,
      "inputs": [
        {"name": "samples", "type": "LATENT", "link": 14},
        {"name": "vae", "type": "VAE", "link": 12}
      ],
      "outputs": [{"name": "IMAGE", "type": "IMAGE", "links": [16], "slot_index": 0}],
      "properties": {"Node name for S&R": "VAEDecode"},
      "title": "VAE Decode"
    },
    {
      "id": 13, "type": "SaveImage",
      "pos": [1650, 400], "size": [315, 270],
      "flags": {}, "order": 12, "mode": 0,
      "inputs": [{"name": "images", "type": "IMAGE", "link": 16}],
      "properties": {"Node name for S&R": "SaveImage"},
      "widgets_values": ["FLUX_dev"],
      "title": "Save Image"
    }
  ],
  "links": [
    [1,  2, 0, 4, 0, "CLIP"],
    [3,  4, 0, 6, 0, "CONDITIONING"],
    [5,  6, 0, 7, 1, "CONDITIONING"],
    [6,  8, 0, 11, 0, "NOISE"],
    [7,  1, 0, 7, 0, "MODEL"],
    [8,  7, 0, 11, 1, "GUIDER"],
    [9,  9, 0, 11, 2, "SAMPLER"],
    [10, 1, 0, 10, 0, "MODEL"],
    [11, 5, 0, 11, 4, "LATENT"],
    [12, 3, 0, 12, 1, "VAE"],
    [13, 10, 0, 11, 3, "SIGMAS"],
    [14, 11, 0, 12, 0, "LATENT"],
    [16, 12, 0, 13, 0, "IMAGE"]
  ],
  "groups": [],
  "config": {},
  "extra": {"ds": {"scale": 0.7, "offset": [0, 0]}},
  "version": 0.4
}

wf_dir = '/content/ComfyUI/user/default/workflows'
os.makedirs(wf_dir, exist_ok=True)
with open(f'{wf_dir}/flux_dev_default.json', 'w') as f:
    json.dump(flux_workflow, f, indent=2)

print("  ✅ Workflow saved: user/default/workflows/flux_dev_default.json")
print("  ✅ Using GGUF Q5 model (works great on T4!)")
print("  ✅ Settings: euler sampler, simple scheduler, 20 steps, guidance 3.5")

# ── Cloudflare Tunnel ──
print("\n📦 Setting up Cloudflare Tunnel...")
if not os.path.exists('/usr/local/bin/cloudflared'):
    subprocess.run(['wget', '-q',
        'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb',
        '-O', '/tmp/cloudflared.deb'], check=True)
    subprocess.run(['dpkg', '-i', '/tmp/cloudflared.deb'], capture_output=True)
    print("  ✅ cloudflared installed")
else:
    print("  ✅ cloudflared already installed")

print("  🔗 Starting tunnel...")
tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:8188'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
)
time.sleep(5)

tunnel_url = None
for _ in range(15):
    line = tunnel.stderr.readline()
    m = re.search(r'(https://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
    if m:
        tunnel_url = m.group(1)
        break

if tunnel_url:
    print(f"\n  {'═' * 56}")
    print(f"  🌐  PUBLIC URL: {tunnel_url}")
    print(f"  {'═' * 56}")
    print(f"  Open once 'Starting server' appears below.\n")
else:
    print("  ⚠️  URL not captured — look for trycloudflare.com in logs.\n")

print("🚀 Launching ComfyUI with FLUX.1 dev (GGUF)...")
print("   GGUF Q5 models work great on T4 — no fp8 hang issues!")
print("   First generation: ~60-90 sec | Subsequent: ~45-60 sec")
print()
print("=" * 60)
print("  📝 HOW TO USE (IMPORTANT - READ THIS!)")
print("=" * 60)
print()
print("  1. Open the trycloudflare.com URL above")
print()
print("  2. LOAD THE FLUX WORKFLOW:")
print("     → Click 'Load' button (right sidebar)")
print("     → Select 'flux_dev_default.json'")
print("     → The FLUX workflow will load with all correct settings")
print()
print("  3. Edit the prompt in the 'Positive Prompt' node")
print()
print("  4. Click 'Queue Prompt' → wait ~60-90 sec")
print()
print("  5. Image appears in 'Save Image' node + saved to output/")
print()
print("=" * 60)

# ── Launch ComfyUI ──
# --lowvram: Offload to CPU as needed for T4's 15GB VRAM
# --preview-method auto: Enable live previews
!python main.py \
    --lowvram \
    --preview-method auto \
    --listen 0.0.0.0 \
    --port 8188

---

## Using ComfyUI with FLUX.1 dev

### First time
1. **Load the workflow first!** Click **Load** (right sidebar) → select **flux_dev_default.json**
2. The FLUX workflow loads with 13 nodes — all settings pre-configured
3. First image takes ~60 sec (CUDA kernel compilation). Subsequent images: ~60–90 sec
4. Edit the **CLIPTextEncode** node text to change your prompt
5. Change **RandomNoise** seed for variety

### Key settings for FLUX.1 dev

| Setting | Value | Notes |
|---------|-------|-------|
| **Steps** | **20** | Quality sweet spot; 25-30 for maximum detail |
| **Guidance** | **3.5** | FLUX's equivalent of CFG. Try 2.5–4.5 |
| **Sampler** | **euler** | Recommended for FLUX |
| **Scheduler** | **simple** | Use `simple` or `beta` for FLUX |
| **Resolution** | **1024×1024** | Native FLUX resolution |
| **Other resolutions** | 1360×768, 768×1360 | Use `EmptySD3LatentImage` node |

### Prompting tips for FLUX.1 dev
- FLUX has **excellent prompt adherence** — be specific and descriptive
- No negative prompts needed for FLUX dev (the model handles anatomy natively)
- Include camera/lens details for photorealism: `85mm lens, f/1.8, Sony A7R IV`
- FLUX can render **text in images** — try `a sign that says "Hello World"`
- Style keywords work well: `cinematic, editorial, studio lighting, golden hour`

### Adjusting resolution (aspect ratios)
In the `EmptySD3LatentImage` node, try these FLUX-friendly resolutions:
| Format | Width | Height |
|--------|-------|--------|
| Square | 1024 | 1024 |
| Landscape | 1360 | 768 |
| Portrait | 768 | 1360 |
| Wide | 1536 | 640 |

### Install extra ComfyUI nodes
1. Click **Manager** in the ComfyUI menu
2. Search for nodes like `ComfyUI-Impact-Pack` (face detailer), `rgthree-comfy`, `ComfyUI-GGUF`
3. Click Install → Restart ComfyUI (re-run Cell 3)

---

## API usage

The ComfyUI API is available at `<your-url>/api`:

```python
import requests, json, copy, time, urllib.request
from PIL import Image
from io import BytesIO

BASE = "https://your-url.trycloudflare.com"

# Load and modify the default workflow
with open('/content/ComfyUI/user/default/workflows/flux_dev_default.json') as f:
    wf = json.load(f)

wf["4"]["inputs"]["text"] = "your custom prompt here"
wf["8"]["inputs"]["noise_seed"] = 12345

prompt_id = requests.post(f"{BASE}/prompt", json={"prompt": wf}).json()["prompt_id"]

# Wait for completion
while True:
    history = requests.get(f"{BASE}/history/{prompt_id}").json()
    if prompt_id in history:
        break
    time.sleep(2)

# Download image
outputs = history[prompt_id]["outputs"]
for node_id, node_output in outputs.items():
    if "images" in node_output:
        for img_info in node_output["images"]:
            img_data = requests.get(f"{BASE}/view",
                params={"filename": img_info["filename"], "subfolder": img_info["subfolder"],
                        "type": img_info["type"]}).content
            Image.open(BytesIO(img_data)).save("flux_output.png")
            print("Saved: flux_output.png")
```

Full API docs: `<your-url>/docs`

---

## Troubleshooting

| Problem | Fix |
|---------|-----|
| **"Out of memory"** | `--lowvram` is already set. Try reducing resolution to 768×768. Factory reset runtime. |
| **Slow generation** | Normal — T4 is not an A100. First image ~60 sec, subsequent ~60–90 sec. |
| **Workflow not loading** | Click **Load** button (right sidebar) → select `flux_dev_default.json` |
| **`UNETLoader` node missing** | Update ComfyUI: re-run Cell 1 with `git pull` in `/content/ComfyUI` |
| **Tunnel URL not showing** | Look in Cell 3 output for `trycloudflare.com` |
| **Black/corrupted images** | VAE issue — ensure `ae.safetensors` downloaded correctly (re-run Cell 2) |
| **T5 encoder errors** | T5 is fp8 and runs on CPU — normal. May see memory warnings, ignore them. |

### Full reset
1. Runtime → **Factory reset runtime**
2. Runtime → Change runtime type → **T4 GPU** → Save
3. Run Cell 1 → Cell 2 → Cell 3

---

## License notice

**FLUX.1 dev** is released under the [FLUX.1-dev Non-Commercial License](https://huggingface.co/black-forest-labs/FLUX.1-dev/blob/main/LICENSE.md). It is free for personal and research use only. For commercial use, see [FLUX.1 pro](https://blackforestlabs.ai/).

---

## Resources

- [ComfyUI GitHub](https://github.com/comfyanonymous/ComfyUI)
- [ComfyUI-Manager](https://github.com/ltdrdata/ComfyUI-Manager)
- [FLUX.1 dev on Hugging Face](https://huggingface.co/black-forest-labs/FLUX.1-dev)
- [ComfyUI Examples (FLUX)](https://comfyanonymous.github.io/ComfyUI_examples/flux/)
- [r/StableDiffusion](https://reddit.com/r/StableDiffusion) — community tips

---

**Made with ❤️ — Star the repo if this saved you time!**